In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [2]:
#Load the data
df = pd.read_pickle("../data/sentence_labels")

#Work on a subset for testing purposes
input_df = df[['Sentence']].head(200).copy()
label_df = df[['Labels']].head(200).copy()



## Formatting the input and labels

In [3]:
# Format the input
from keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer
import torch

Using TensorFlow backend.


In [4]:
"""
Prepares the sentences and labels for use with BERT-type models.

"""

def format_data(input_df, label_df):
    #Load swedish tokenizer
    tokenizer = AutoTokenizer.from_pretrained("KB/bert-base-swedish-cased-ner")
    
    # Use WORDPIECE tokenization
    #input_df['Tokenized'] = input_df[['Sentence']].apply(lambda x: tokenizer.tokenize(x[0]), axis=1)
    # Ensure no named entities have been split apart

    # use WORD tokenization
    input_df['Tokenized'] = input_df[['Sentence']].apply(lambda x: x[0], axis=1)
    label_df['Tokenized'] = label_df[['Labels']].apply(lambda x: x[0], axis=1)

    # Replace words with integers, add the special [CLS] and [SEP] tokens
    input_df['Integerized'] = input_df[['Tokenized']].apply(lambda x: tokenizer.encode(x[0], add_special_tokens=True), axis=1)
    label_df['Integerized'] = label_df[['Tokenized']].apply(lambda x: tokenizer.encode(x[0], add_special_tokens=True), axis=1)

    # Pad and truncate all sentences so they are the same length
    length = 50
    input_df['Input'] = input_df[['Integerized']].apply(lambda x: pad_sequences(x, maxlen=length, dtype="long", truncating="post", padding="post")[0], axis=1)
    label_df['Output_Class'] = label_df[['Integerized']].apply(lambda x: pad_sequences(x, maxlen=length, dtype="long", truncating="post", padding="post")[0], axis=1)

    # Create attention mask. Attention is 0 for padding, else 1
    input_df['Attention_Mask'] = input_df[['Input']].apply(lambda x: (x[0] != 0).astype(int), axis=1)
    
    return input_df, label_df

input_df, label_df = format_data(input_df, label_df)

In [5]:
#Sanity checks for Labels
def check_labels(index):
    for column in label_df:
        print(column)
        print(label_df[[column]].iloc[index][0])
        print(f"Length: {len(label_df[[column]].iloc[index][0])}")
        print()
    
# check_labels(0)

In [6]:
#Sanity checks for sentences
def check_sentence(index):
    for column in input_df:
        print(column)
        print(input_df[[column]].iloc[index][0])
        print(f"Length: {len(input_df[[column]].iloc[index][0])}")

        print()
    
# check_sentence(0)

In [7]:
# To prepare a single sentence
index = 0

#Convert lists torch tensors
data_tensor = torch.tensor(input_df[['Input']].iloc[index][0]).unsqueeze(0)
mask_tensor = torch.tensor(input_df[['Attention_Mask']].iloc[index][0]).unsqueeze(0)

#The tags don't need to be a tensor since the labels will be used with the linear classifier outside ALBERT
tags_output = label_df[['Output_Class']].iloc[index][0]

In [8]:
# To prepare all sentences

# Convert the list of lists into tensors
data_matrix = torch.tensor([x[0] for x in input_df[['Input']].values])
mask_matrix = torch.tensor([x[0] for x in input_df[['Attention_Mask']].values])

#This doesn't need to be a tensor since the labels will be used with the linear classifier outside ALBERT
tags_matrix = np.array([x[0] for x in label_df[['Output_Class']].values])


## Generate Embeddings

In [9]:
#Load model
from transformers import AutoModel
model = AutoModel.from_pretrained('KB/bert-base-swedish-cased')

In [10]:
"""
This function takes a single input sentence and mask and generates an embedding for all the tokens in the sentence, using the cpu
"""
def get_embeddings_with_cpu(data_tensor, mask_tensor):
    embeddings = model.forward(input_ids=data_tensor,
        attention_mask=mask_tensor,
        head_mask=None)
    print(embeddings[0].shape)
    
    return embeddings[0]
#embedding_matrix = get_embeddings_with_cpu(data_matrix[:10], mask_matrix[:10])


In [11]:
"""
For dealing with large amounts of data, a GPU is much faster

The resulting embedding matrix is a three-dimensional tensor corresponding to [Sentence][Words][Embeddings]
embeddings[5][4][:] is thus the embedding of the fourth word in the fifth sentence
"""

def get_embeddings_with_gpu(data_matrix, mask_matrix):
    # Load the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #print(f"Using device: {torch.cuda.get_device_name(0)}")

    # Set the model to use the device
    model.cuda()

    # Move the data onto the GPU
    data_matrix = data_matrix.to(device)
    mask_matrix = mask_matrix.to(device)

    # Generate embeddings
    matrix_embedding = model.forward(input_ids=data_matrix,
        attention_mask=mask_matrix,
        head_mask=None)[0]
    #print(f"Embedding generated with shape {batch_embedding.shape}")

    # Make it an ordinary np array instead of a torch
    matrix_embedding = np.array(matrix_embedding.tolist())

    return matrix_embedding

#embedding_matrix = get_embeddings_with_gpu(data_matrix[:10], mask_matrix[:10])


In [ ]:
"""
Most people won't be able to load all the data onto the GPU at once however, so it's better to do it in batches.
(50 input sentences take 2803MB on my computer, for example).

This method batchifies and stitches together the batches 
"""
def get_embeddings_with_gpu_batch(data_matrix, mask_matrix, batch_size):
    num_items = data_matrix.shape[0]
    num_loops = int(np.ceil(num_items/batch_size))
    
    start = 0
    end = batch_size
    data_holder = []
    
    for i in trange(num_loops):        
        # Split the data into batches
        data_batch = data_matrix[start:end]
        mask_batch = mask_matrix[start:end]
        
        #Get the embedding for the batch
        batch_embedding = get_embeddings_with_gpu(data_batch, mask_batch)

        data_holder.append(batch_embedding)
        
        #Move to next batch
        start += batch_size
        end += batch_size
    
    # Merge the batches we've generated
    embedding_matrix = np.vstack(data_holder)

    print(f"Final embedding generated with shape {embedding_matrix.shape}")

    return embedding_matrix


embedding_matrix = get_embeddings_with_gpu_batch(data_matrix, mask_matrix, 50)
print(embedding_matrix.shape)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Right now our data is grouped by sentence. Since we'll do classification on a token-level however,
# we join the sentences to get a list of tokens and their embeddings
input_data = embedding_matrix.reshape(embedding_matrix.shape[0]*embedding_matrix.shape[1],-1)

input_data.shape

# Also, for convenience we create a token-embedding mapping
cols = ["Token", "Embedding"]
token_embedding_df = pd.DataFrame(columns=cols)
token_embedding_df

In [ ]:
#token_embedding_df['Token'] = input_df[['Sentence']].apply(lambda x: x, axis=0)

#token_embedding_df

In [ ]:
# And of course, we do the same for the labels

tags_matrix = np.array([x[0] for x in label_df[['Output_Class']].values])
print(tags_matrix.shape)

#Since we are dealing with token-level classification, break, unravel the sentence grouping we have
token_labels = tags_matrix.reshape(-1,1)
token_labels = pd.DataFrame(token_labels)

#Since we are dealing with a multiclass problem, switch to one-hot encoding
one_hot_token_labels = pd.get_dummies(token_labels[0])

one_hot_token_labels

In [ ]:
# Some last checks on the data
num_dimensions = input_data.shape[1]
num_classes = one_hot_token_labels.shape[1]
print(num_dimensions,num_classes)

In [ ]:
# For conveniece, let's save the generated embeddings so we can skip the training process

In [ ]:
split

## Split into Train and Test

In [ ]:
ratio = 0.8

split = int(np.ceil(0.8*input_data.shape[0]))

train_x = input_data[:split]
train_y = one_hot_token_labels[:split]

test_x = input_data[split:]
test_y = one_hot_token_labels[split:]

## Train the classifier on the embeddings

In [ ]:
# Define a simple baseline model
from keras.models import Sequential
from keras.layers import Dense
def baseline_model():
    # create model
    model = Sequential()
    model.add(
        Dense(8, input_dim=num_dimensions, activation='relu'))
    model.add(
        Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [ ]:
model = baseline_model()

In [ ]:
# Train the model, setting aside 20% data for validation
history = model.fit(
    train_x,
    train_y,
    epochs=20,
    batch_size=10,
    validation_split=0.2)

print('\nhistory dict:', history.history.keys())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

training_df = pd.DataFrame()
training_df['Accuracy'] = history.history['accuracy']
training_df["Validation Accuracy"] = history.history['val_accuracy']
training_df['Loss'] = history.history['loss']
training_df["Val Loss"] = history.history['val_loss']


from datetime import date

HISTORY_NAME = "history_" + str(date.today()) + "_epochs_" +str(training_df.shape[0])
HISTORY_NAME

# Save the history object for posterity
training_df.to_pickle('../data/' + HISTORY_NAME)

sns.set()
fig, ax = plt.subplots(figsize=(15,10))
sns.lineplot(data = training_df, ax = ax)
plt.ylabel('Metric')
plt.xlabel('Epoch')
plt.show()

In [ ]:
# Visualize training history

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
#estimator = KerasClassifier(build_fn=baseline_model, epochs=20, batch_size=5, verbose=0)

In [ ]:
from sklearn.model_selection import KFold

# Use 10 fold cross-validation to reduce risk of bias
#kfold = KFold(n_splits=2, shuffle=True)

In [ ]:
from sklearn.model_selection import cross_val_score

#results = cross_val_score(estimator, train_x, train_y, cv=kfold, verbose=1)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# Evaluate

In [ ]:
#print('\n# Evaluate on test data')
#results = model.evaluate(test_x, test_y)
#print('test loss, test acc:', results)


In [ ]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(test_x[:3])
print('predictions shape:', predictions.shape)

## Test predictions

In [ ]:
"""
This method takes a sentence and does NER on it
"""
def NER(sentence):
    #Sentence
    s = sentence
    print(f"Sentence: {s}")
    
    #Tokens
    t = tokenizer.encode(s, add_special_tokens=True)
    print(f"Integer representation: {t}")
    
    #Padded
    p = pad_sequences([t], maxlen=length, dtype="long", truncating="post", padding="post")
    
    #Mask
    m = (p[0] != 0).astype(int)
    
    #Tensors
    tp = torch.tensor(p)
    tm = torch.tensor(m)
    
    print(f"Embedding: {tm}")
    
    #Encoding
    e = get_embeddings_with_cpu(tp, tm)
    print(f"Embedding: {tp}")
    
s = "Hej, Nynäshamn är min stad"
NER(s)